# ACS Cost Burden for Renters and Homeowners in Phoenix Urban Villages

- https://www.census.gov/data/developers/data-sets/acs-5year.html

For income as a percentage of rent, or owner costs for with and without a mortgage
- https://api.census.gov/data/2010/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_acs as get
import utilcalcs as calc
import geo_agg
from acs_cost_vars import *

In [3]:
bgp_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bgp_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')
for df in [bgp_10,bgp_20]: df.geoid = df.geoid.apply(lambda x: '{0:0>12}'.format(x))
    
#get ride of area & geo stuff not being useed
bgp_20 = bgp_20.drop(['aland20','awater20','lat20','lon20','land_acre'],axis=1)
bgp_10 = bgp_10.drop(['aland10','awater10','lat10','lon10','land_acre'],axis=1)

In [4]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
y0 = '2013'

cols_rent = 'group(B25070)'
cols_own = 'group(B25091)'

source = 'acs/acs5'

rent_vars = ['GEO_ID']+rent_vars  #imported from acs vars
own_vars = ['GEO_ID']+own_vars    #"

##  RENT BURDEN 

In [5]:
def group_rent(df,year):
    df['r_u30_E'] = df.loc[:,r_u30_e].sum(axis=1)
    df['r_u30_M'] = df.apply(lambda x: (calc.get_moe(x[r_u30_m])),axis=1)
    df['r_3050_E'] = df.loc[:,r_3050_e].sum(axis=1)
    df['r_3050_M'] = df.apply(lambda x: (calc.get_moe(x[r_3050_m])),axis=1)
    df = df.rename({'B25070_001E':'r_tot_E','B25070_001M':'r_tot_M',
                    'B25070_010E':'r_o50_E','B25070_010M':'r_o50_M',
                    'B25070_011E':'r_na_E', 'B25070_011M':'r_na_M'},axis=1)
    
    df = df.drop(r_u30_e+r_u30_m+r_3050_e+r_3050_m,axis=1)
    
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

In [6]:
def rent_uvil(geo_df,year):
    df = get.get_bgp(source,year,cols_rent)
    df = get.clean_data(df,rent_vars)
    df = group_rent(df,year)
    df = pd.merge(geo_df,df,how='left',left_on='geoid',right_on='GEO_ID')
    df = df.drop(['geoid','GEO_ID'],axis=1)
    df = geo_agg.sumgeo_cv(df,'name')
    return df

In [16]:
def rent_change(df,change_vars,year1,year0):
    for var in change_vars:
        df[f'r_{var}_{year0[-2:]}{year1[-2:]}E'] = df[f'r_{var}_{year1[-2:]}E'] - df[f'r_{var}_{year0[-2:]}E']
        df[f'r_{var}_{year0[-2:]}{year1[-2:]}M'] = df.apply(lambda x: (calc.get_moe([x[f'r_{var}_{year0[-2:]}M'],\
                                                                                x[f'r_{var}_{year1[-2:]}M']])),axis=1)
        df[f'r_{var}_{year0[-2:]}{year1[-2:]}C'] = df.apply(lambda x: (calc.get_cv(x[f'r_{var}_{year0[-2:]}{year1[-2:]}E'],\
                                                                            x[f'r_{var}_{year0[-2:]}{year1[-2:]}M'])),axis=1)
        for  y  in [year0[-2:],year1[-2:]]:
            df[f'r_{var}_{y}P'] = df[f'r_{var}_{y}E'] / df[f'r_tot_{y}E']
        
    return  df

####  Urban Villages

In [17]:
dfY1 = rent_uvil(bgp_20,y1)
dfY0 = rent_uvil(bgp_10,y0)
df_uv = pd.merge(dfY0,dfY1,on='name',how='left')
df_uv = rent_change(df_uv,['u30','3050','o50','na'],y1,y0)

In [19]:
df_uv.head()

,name,r_3050_13E,r_3050_13M,r_3050_13C,r_u30_13E,r_u30_13M,r_u30_13C,r_o50_13E,r_o50_13M,r_o50_13C,...,r_o50_1321E,r_o50_1321M,r_o50_1321C,r_o50_13P,r_o50_21P,r_na_1321E,r_na_1321M,r_na_1321C,r_na_13P,r_na_21P
0,Laveen,1033.0,293.600068,17.277859,1449.0,304.205523,12.762413,1132.0,281.465806,15.115180,...,-74.0,380.267011,312.385617,0.294332,0.247023,-123.0,168.949697,83.499986,0.060322,0.025449
1,Deer Valley,5979.0,623.671388,6.341053,11736.0,921.534047,4.773373,4496.0,497.294681,6.723906,...,766.0,866.019630,68.727898,0.193660,0.198252,125.0,465.665116,226.463278,0.043289,0.042574
2,Maryvale,7943.0,687.452544,5.261290,10756.0,800.092495,4.521927,9413.0,710.069011,4.585710,...,-2331.0,1005.710197,26.227970,0.311122,0.223541,-603.0,535.111203,53.946196,0.070831,0.048610
3,North Mountain,7054.0,696.894540,6.005729,12834.0,917.155930,4.344254,6782.0,635.919806,5.700050,...,835.0,1008.883046,73.449433,0.238610,0.240322,48.0,562.500667,712.386863,0.061675,0.056823
4,Paradise Valley,5307.0,588.318791,6.739035,11917.0,891.158235,4.545922,5205.0,576.446008,6.732432,...,95.0,946.788255,605.847548,0.220401,0.220081,-151.0,447.912938,180.322848,0.050263,0.043020


#### City of Phoenix

In [20]:
def make_phx(year):
    df = get.get_phx(source,year,cols_rent)
    df = get.clean_data(df,rent_vars)
    df = group_rent(df,year)
    return df

In [21]:
df1 = make_phx(y1)
df0 = make_phx(y0)

In [22]:
df_phx = pd.merge(df0,df1,on='GEO_ID',how='left')
df_phx = geo_agg.sumgeo_cv(df_phx,'GEO_ID') #calc cvs
df_phx =  rent_change(df_phx,['u30','3050','o50','na'],y1,y0)

In [24]:
df_phx.to_excel('output/phx_rent.xlsx',index=False)

## OWNER BURDEN